<a href="https://colab.research.google.com/github/salmanahmad10/BBAK/blob/main/Segmentation_Model_DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Preprocessing Module**

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

In [ ]:
H = 128
W = 128

In [ ]:
def process_data(data_path, file_path):
  df = pd.read_csv(file_path, header=None)
  names = df[0].values

  images = [os.path.join(data_path, f"Images/{name}") for name in names]
  masks = [os.path.join(data_path, f"SegmentedImages/{name}") for name in names]

  return images, masks


In [ ]:
def load_data(path):
  train_valid_path = os.path.join(path, "Images/Train.txt")
  test_path = os.path.join(path, "SegmentedImages/Test.txt")

  train_x, train_y = process_data(path, train_valid_path)
  test_x, test_y = process_data(path, train_valid_path)

  train_x, valid_x = train_test_split(train_x, test_size = 0.1, random_state = 42)
  train_y, valid_y = train_test_split(train_y, test_size = 0.1, random_state = 42)

  return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path = "gdrive/MyDrive/master/SharedGoogleDrive/Dataset/HandDrawnWireframes/Annotation/"

In [ ]:
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(path)

print(f"Dataset: Train: {len(train_x)} - Valid: {len(valid_x)} - Test: {len(test_x)}")

Dataset: Train: 202 - Valid: 23 - Test: 225


In [ ]:
def readImage(x):
  x = cv2.imread(x, cv2.IMREAD_COLOR)
  x = cv2.resize(x, (W, H))
  x = x / 255.0
  x = x.astype(np.float32)

  return x

def readMask(x):
  x = cv2.imread(x, cv2.IMREAD_GRAYSCALE)
  x = cv2.resize(x, (W, H))
  x = x / 255.0
  x -= 1
  x = x.astype(np.int32)

  return x

In [ ]:
def tf_database(x, y, batch=8):
  dataset = tf.data.Dataset.from_tensor_slices((x,y))
  dataset = dataset.shuffle(buffer_size=5000)
  dataset = dataset.map(preprocess)
  dataset = dataset.batch(batch)
  dataset = dataset.repeat()
  dataset = dataset.prefetch(2)
  return dataset

In [ ]:

def preprocess(x, y):
  def f(x, y):
    x = x.decode()
    y = y.decode()

    image = readImage(x)
    mask = readMask(y)

    return image, mask

  image, mask = tf.numpy_function(f, [x, y], [tf.float32, tf.int32])
  mask = tf.one_hot(mask, 12, dtype=tf.int32)
  image = tf.ensure_shape(image, [H, W, 12])
  mask = tf.ensure_shape(mask, [H, W, 12])

  return image, mask

In [ ]:
 (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(path)
 print(f"Dataset: Train: {len(train_x)} - Valid: {len(valid_x)} - Test: {len(test_x)}")


Dataset: Train: 202 - Valid: 23 - Test: 225


In [ ]:
#dataset = tf_dataset(train_x, train_y, batch=8)
#for x, y in dataset:
#  print(x.shape, y.shape) ## (8, 256, 256, 3), (8, 256, 256, 3)

NameError: ignored